In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve
from yellowbrick.target import ClassBalance
import seaborn as sns
import itertools
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import OrdinalEncoder
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [103]:
# Load the data and create a copy to use to verify preprocessing steps
quicken_df = pd.read_csv('data/quicken_2019.csv')
check_df = quicken_df.copy()
# Pull Census Tract dictionary from different notebook
%store -r FIPS_dict

# Step 1: Condense High Cardinality Variables
Action Taken: 
* 1: Loan originated
* 2: Application approved but not accepted
* 3: Application denied
* 4: Application withdrawn by applicant
* 5: File closed for incompleteness
* 6: Purchased loan
* 7: Preapproval request denied
* 8: Preapproval request approved but not accepted

Interpreting which actions are 'approved:' 
1, 2, 6

Denied:
3

Irrelevant to model: 4, 5, 7, 8

-------
* Race and Ethnicity columns both allow the applicant to report a more specific value. For example, an applicant can select Asian (2) or more specifically Chinese(22), etc. Because this column will be onehotencoded, I'm going to simplify these columns to the top-level categories:
    - American Indian or Alaska Native
    - Asian
    - Black or African American
    - Native Hawaiian or Other Pacific Islander
    - White
    - Hispanic
    - Non-Hispanic
------------

Census Tract
* The Census Tract column contains ~65K unique values, 366 of which are NA. These were mapped to 10 'primary codes' which indicate if a region is rural, urban, metropolitan, etc. However, because I couldn't use the mapper dictionary if null values were present, dropped these in this preprocessing step rather than in the general missing values section, below. 

In [104]:
# action taken
quicken_df = quicken_df[~quicken_df['action_taken'].isin([4,5,7,8])]
quicken_df['action_taken'] = quicken_df['action_taken'].map(lambda x: 0 if x ==3 else 1)

# applicant race 
def condense_race(x): 
    if x == 6 or x == 7:
        x = np.nan
    if x> 10 and x < 20: 
        x = 1
    if x> 20 and x < 30: 
        x = 2
    if x> 40 and x < 50: 
        x = 4
    else: 
        x = x
    return x

# applicant ethnicity
def condense_ethnicity(x): 
    if x == 3 or x == 4: 
        x = np.nan
    if x !=2: 
        x = 1
    else: 
        x = x
    return x 

def condense_sex(x): 
    if x == 3 or x == 4: 
        x = np.nan
    else: 
        x = x
    return x 

In [105]:
quicken_df['applicant_race-1'] = quicken_df['applicant_race-1'].apply(lambda x: condense_race(x))
quicken_df['applicant_ethnicity-1'] = quicken_df['applicant_ethnicity-1'].apply(lambda x: condense_ethnicity(x))
quicken_df['applicant_sex'] = quicken_df['applicant_sex'].apply(lambda x: condense_sex(x))

quicken_df = quicken_df.dropna(subset=['census_tract'])
quicken_df['census_tract'] = quicken_df['census_tract'].astype(int)
quicken_df['census_tract'] = quicken_df['census_tract'].astype(str)
quicken_df['census_tract'] = quicken_df['census_tract'].map(FIPS_dict)

# Step 2: Feature Engineering
* Planned to add the following columns: 
    - has co-signer
    - bi/multi-racial
    - co-signer bi/multi-racial
    
However, FFEIC updated their datasets with new "derived" variables, which already achieve this goal for race, ethnicity, and sex. These values don't add up to the original field. Will use both for now. 

* Ordinal Encoding for Age column
* DIR contains str and int, mapped using dictionary

In [106]:
# Create ordinalencoder object
enc = OrdinalEncoder()

# Convert column to array
age_df = quicken_df[['applicant_age']].to_numpy()
enc.fit(age_df)
enc.categories_

# fitted encoder to array
enc_age_arr = enc.transform(quicken_df[['applicant_age']].to_numpy())

# Apply encoder transformation
quicken_df['applicant_age'] = enc_age_arr

In [107]:
DIR_mapper = {'20%-<30%': 25, '30%-<36%': 33, '>60%': 65, '<20%': 15, '50%-60%': 55}


quicken_df['debt_to_income_ratio'] = quicken_df['debt_to_income_ratio'] \
                                    .map(lambda x: DIR_mapper[x] if x in DIR_mapper else int(x), na_action='ignore')

# Step 3: Missing Values
1. Some variables are perfect predictors when they are null. Filled all nan values with 99 so I could run a crosstabulation to check for this. 

In [108]:
quicken_df['income'] = quicken_df['income'].fillna(quicken_df['income'].median())

In [109]:
quicken_df = quicken_df.fillna(99)


# Step 4: Feature Selection
1. Drop columns containing only one unique value
Null values are represented differently for each variable in the dataset. Looked at each variable and recoded null values to 99 where applicable
2. Check for perfect predictors and drop those
3. Drop certain columns based on what information is relevant. This will apply to all lenders
    a. ```state_code``` and ```county_code``` - using census tract and ```derived_msa-md``` because they are more informative
4. ```Income``` converted na back to np.nan and will impute median for this column
5. There are several census features that were appended to the dataset after the lender submitted their report. While these features might be used in a different part of this study, they will be excluded from the dataset that will be fed into the model, as the bank did not report these values. 
6. Exclude 2nd lien mortgage, interest only payment loans (only ~400 of these loans)

In [8]:
perfect_predictors = []
for col in quicken_df.columns: 
    cross = pd.crosstab(quicken_df[col], quicken_df['action_taken'])
    if cross[0].sum() == 0 or cross[1].sum() == 0:
        perfect_predictors.append(col)
perfect_predictors

[]

In [110]:
one_unique_value_lst = []
for col in quicken_df.columns:
    if len(quicken_df[col].unique()) ==1: 
        one_unique_value_lst.append(col)
quicken_df = quicken_df.drop(one_unique_value_lst, axis=1)

In [147]:
# null coded as 99999, 0, 5 --> recode to 99
quicken_df['derived_msa-md'] = quicken_df['derived_msa-md'].apply(lambda x: 99 if x == 99999 else x)
quicken_df['purchaser_type'] = quicken_df['purchaser_type'].apply(lambda x: 99 if x == 0 else x)
quicken_df['loan_purpose'] = quicken_df['loan_purpose'].apply(lambda x: 99 if x == 5 else x)
quicken_df['applicant_credit_score_type'] = quicken_df['applicant_credit_score_type'] \
                                                    .apply(lambda x: 99 if x == 9 else x)
quicken_df['applicant_ethnicity_observed'] = quicken_df['applicant_ethnicity_observed'] \
                                                    .apply(lambda x: 99 if x == 3 else x)
quicken_df['applicant_race_observed'] = quicken_df['applicant_race_observed'] \
                                                    .apply(lambda x: 99 if x == 3 else x)
quicken_df['submission_of_application'] = quicken_df['submission_of_application'] \
                                                    .apply(lambda x: 99 if x == 3 else x)

In [169]:
# Columns to drop b/c irrelevant or class imbalance is extreme 
drop_irrelevant = ['state_code', 'county_code', 'open-end_line_of_credit', 'manufactured_home_land_property_interest', 
                  'manufactured_home_secured_property_type', 'co-applicant_credit_score_type', 'applicant_ethnicity-2',
                  'applicant_ethnicity-3', 'applicant_ethnicity-4', 'co-applicant_ethnicity-1', 'co-applicant_ethnicity-2',
                  'co-applicant_ethnicity-3', 'co-applicant_ethnicity-4', 'co-applicant_ethnicity_observed',
                  'applicant_race-2', 'applicant_race-3', 'applicant_race-4', 'applicant_race-5', 
                  'co-applicant_race-1', 'co-applicant_race-2', 'co-applicant_race-3', 'co-applicant_race-4', 
                  'co-applicant_race-5', 'co-applicant_race_observed', 'co-applicant_sex', 'applicant_sex_observed',
                  'co-applicant_sex_observed', 'co-applicant_age', 'applicant_age_above_62', 'co-applicant_age_above_62',
                  'submission_of_application', 'derived_loan_product_type', 'interest_only_payment']

confirmed_perfect_pred = ['denial_reason-1', 'denial_reason-2', 'denial_reason-3', 'purchaser_type', 'preapproval', 
                       'interest_rate', 'rate_spread', 'hoepa_status', 'total_loan_costs', 'origination_charges', 
                       'discount_points', 'lender_credits', 'initially_payable_to_institution']

census_appended = ['tract_minority_population_percent', 'tract_population', 'ffiec_msa_md_median_family_income',
                  'tract_to_msa_income_percentage', 'tract_owner_occupied_units', 'tract_one_to_four_family_homes',
                  'tract_median_age_of_housing_units']



In [197]:
quicken_df = quicken_df.drop(drop_irrelevant + confirmed_perf_pred + census_appended, axis=1)

In [183]:
# Subselections of relevant variables
quicken_df = quicken_df[quicken_df['lien_status'] == 1]

# Examine Class Imbalance


In [14]:
boa_df['action_taken'].value_counts()

NameError: name 'boa_df' is not defined

# Examing Colinearity

# Check for Perfect Predictors